In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('cleand_tweets.csv')
df.shape

(71875, 24)

In [8]:
df = pd.read_csv('tweets_translated.csv')
df['tweet']

0        Between this, another brilliant championship r...
1        The new Republican formula: 1) Proclaim yourse...
2        @MNofroni @trienegaming @F1 It actually is the...
3        @twoandahalfapen @impartialf1fan @F1 @Max33Ver...
4        @LandoNorris  @McLarenF1 @F1 new t-shirt has a...
                               ...                        
71870    1968: Born Ricardo Rosset, Brazilian Formula 1...
71871    Shitty no one knows how to race with formula 1...
71872    ((The Most Read)) #Noticias #mundotuerca #form...
71873    💖🐶 'Fabbio' brings all the attitude of formula...
71874    @marca @Carlossainz55 RIDICULOUS .... that's h...
Name: tweet, Length: 71875, dtype: object

In [24]:
df.isnull().sum(axis=0)

(71874, 23)

In [12]:
#Droping all coulumns that are empty or with many missing values.
df = df.drop(['thumbnail'], axis=1)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71874 entries, 0 to 71874
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               71874 non-null  int64 
 1   date             71874 non-null  object
 2   time             71874 non-null  object
 3   user_id          71874 non-null  int64 
 4   username         71874 non-null  object
 5   name             71873 non-null  object
 6   tweet            71874 non-null  object
 7   language         71874 non-null  object
 8   mentions         71874 non-null  object
 9   urls             71874 non-null  object
 10  photos           71874 non-null  object
 11  replies_count    71874 non-null  int64 
 12  retweets_count   71874 non-null  int64 
 13  likes_count      71874 non-null  int64 
 14  hashtags         71874 non-null  object
 15  cashtags         71874 non-null  object
 16  link             71874 non-null  object
 17  retweet          71874 non-null

In [26]:
df = df.drop_duplicates(subset='id')

In [27]:
df.shape

(71874, 23)

In [28]:
len(df['id'].unique())

71874

## Cleaning Tweets 

In [29]:
def hashtag_removal(r):
    """
    function to remove hashtags from a tweet
    """
    twt=r['tweet'].lower()
    tags=list(r['hashtags'][1:-1].split(','))
    for i in tags:
        i=i.strip()
        i=i[1:-1]
        twt=twt.replace('#'+i,'')
    return twt 
df['clean_tweet']=df.apply(lambda x: hashtag_removal(x),axis=1)

In [30]:
!pip install tweet-preprocessor
import preprocessor as p
#Clening the tweets using tweet-preprocesor
df['clean_tweet']=df['clean_tweet'].apply(lambda x:p.clean(x))

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [31]:
import regex as re
def punctuation_removal(r):
    """
    Removing punctioion from tweet
    """
    ls=re.findall(r'&(\w+);', r)
    for i in ls:
        r=r.replace(f'&{i};','')
    punc = '’’!()-[]{};:’”\,<>./?@#$%^&*_~’’' 
    for ele in r:
         if ele in punc:
            r = r.replace(ele, '')
         return r
df['clean_tweet']=df['clean_tweet'].apply(lambda x:punctuation_removal(x))

In [32]:
#Remove Digits
df['clean_tweet'] = df['clean_tweet'].astype(str).str.replace(' \d+', '')
df['clean_tweet']

0        between this, another brilliant championship r...
1        the new republican formula: ) proclaim yoursel...
2        it actually is the constructors championship. ...
3        its litteraly turn and perfectly fine. didnt l...
4                      new t-shirt has arrived and love it
                               ...                        
71870       born ricardo rosset, brazilian formula driver.
71871    shitty no one knows how to race with formula ....
71872    the most read)) bentley celebrates years of co...
71873        'fabbio' brings all the attitude of formula .
71874        ridiculous .... that's how it goes in formula
Name: clean_tweet, Length: 71874, dtype: object

## Tokenization

In [33]:
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')
from nltk.tokenize import TweetTokenizer
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
 return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
df['tokenized_tweet'] = df['clean_tweet'].apply(lambda x:lemmatize_text(x))
df['tokenized_tweet']

[nltk_data] Downloading package wordnet to /home/repl/nltk_data...
[nltk_data] Downloading package stopwords to /home/repl/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /home/repl/nltk_data...


0        [between, this, ,, another, brilliant, champio...
1        [the, new, republican, formula, :, ), proclaim...
2        [it, actually, is, the, constructor, champions...
3        [it, litteraly, turn, and, perfectly, fine, .,...
4               [new, t-shirt, ha, arrived, and, love, it]
                               ...                        
71870    [born, ricardo, rosset, ,, brazilian, formula,...
71871    [shitty, no, one, know, how, to, race, with, f...
71872    [the, most, read, ), ), bentley, celebrates, y...
71873    [', fabbio, ', brings, all, the, attitude, of,...
71874    [ridiculous, ..., that's, how, it, go, in, for...
Name: tokenized_tweet, Length: 71874, dtype: object

**Stop Words removal**

In [34]:
stop_words = set(stopwords.words('english'))
df['tokenized_tweet'] = df['tokenized_tweet'].apply(lambda x: [item for item in x if item not in stop_words])

In [35]:
df = df.sort_values('date')
df.shape
df['tweet'][0]

'Between this, another brilliant championship run from Max in the Red Bull, the Danny Ricciardo Hulu show, the Apple Lewis Hamilton doc, the Joseph Kosinski / Brad Pitt film ( and yah DTS)… WE ARE LIVING IN THE GOLDEN AGE OF FORMULA 1  https://t.co/x68a1NrZgQ'

In [36]:
df.to_csv('cleand_tweets.csv', index=False)